# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [34]:
%%capture

#Instalando o tweepy
!pip install tweepy
#!pip install -U textblob
!pip install sklearn-pandas

#python -m textblob.download_corpora



Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [128]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from nltk.tokenize import word_tokenize

import numpy as np
#import nltk
#nltk.download()
from nltk.corpus import stopwords

import nltk

import re

from collections import Counter

import string

from nltk.tokenize.moses import MosesTokenizer, MosesDetokenizer


#from textblob import TextBlob
#from textblob.classifiers import NaiveBayesClassifier



___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@tainarasm98***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @tainarasm98

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])



___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = "Ruffles"

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'



Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)



Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))
    
    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
"""xls_file = pd.ExcelFile('Ruffles.xlsx')

df = xls_file.parse('Treinamento')




relevante = pd.DataFrame({'Treinamento': df['Treinamento'],
                          'Classificação': df['Classificação']})

irrelevante = pd.DataFrame({'Treinamento': df['Treinamento'],
                          'Classificação': df['Classificação']})

relevante = relevante[(df['Classificação']=='Relevante')]

relevante['Treinamento'].apply(str)
#relevante['Treinamento'] = relevante['Treinamento'].astype('category')
print(relevante.dtypes)
#relevante = [w for w in relevante['Treinamento'] if not w in stopwords.words("portuguese")]


treino = df['Treinamento']
irrelevante = irrelevante[(df['Classificação']=='Irrelevante')]



df['Treinamento'] = df['Treinamento'].str.strip()
df['Treinamento'] = df['Treinamento'].str.lower().str.split()

count_vect = CountVectorizer(analyzer="word")
freq_tweets = count_vect.fit_transform(treino)
modelo = MultinomialNB()
modelo.fit(freq_tweets.treino)

"""for i in range(len(relevante['Treinamento'])):
    treinamento_counts = count_vect.fit_transform(df['Treinamento'][i])
    treinamento_counts.shape 
    count_vect.vocabulary_.get(u'algorithm')
    #print(count_vect.vocabulary_)"""


#a = Counter(df['Treinamento'].str.split())
#print(a)


#d['mynewkey'] = 'mynewvalue'


#a = Counter(df['Treinamento'][0])
#print(a)


relevante = df['Classificação'].value_counts().reindex(['Relevante', 'Irrelevante'])

treinamento = df['Treinamento']


print()
#stop words - para limpar

#print (stopwords.words("portuguese")) 

#for i in range(len(df['Treinamento'])):
    #if df['Treinamento'][i] in stopwords.words('portuguese'):
        #print('sim')
        #df['Treinamento'][i] in df['Treinamento'][i]"""
        
        

    


SyntaxError: invalid syntax (<ipython-input-7-a15cf7e8fad9>, line 35)

In [23]:
xls_file = pd.ExcelFile('Ruffles.xlsx')

df = xls_file.parse('Treinamento')
df2 = xls_file.parse('Teste')


treinamento = pd.DataFrame({'Treinamento': df['Treinamento']})
treinamento = treinamento['Treinamento']
stop = stopwords.words('portuguese')
#stops = str(stops)
remove_list = []

"""for line in treinamento['Treinamento']:
    for w in line.split():
        if w.lower() in stops:
            if w.lower() not in remove_list:
                remove_list.append(w)"""
      
treinamento = list(treinamento)   
""""""for frase in treinamento.split():
    print(frase)
    for palavra in frase:
        if palavra.lower() in stopwords.words('portuguese'):
            treinamento = treinamento.remove(palavra)
    

print(treinamento)
#print(treinamento['Treinamento'])
#for i in treinamento['Treinamento'].str.split():
    #for j in i:
        #if j in remove_list:
            #treinamento['Treinamento'] = treinamento['Treinamento'].str.split(j)


#print(treinamento['Treinamento'])    

remove_list = str(remove_list)
#remove_list.split()

#['Treinamento'] = treinamento['Treinamento'].apply(lambda x: [item for item in string.split(x) if item not in stop])
#treinamento['Treinamento'] = treinamento['Treinamento'].str.strip().str.split()
df['Treinamento'] = df['Treinamento'].str.lower()




#treinamento['Treinamento'] = treinamento['Treinamento'].str.strip().str.lower().str.split()



"""for i in range(len(treinamento['Treinamento'])):
    if treinamento['Treinamento'][i] not in stops:
        treinamento['Treinamento'][i]in treinamento['Treinamento'][i]"""


            
            

            
#print(treinamento['Treinamento'])


#filteredtext = [t for t in treinamento['Treinamento'] if t not in stopwords.words('portuguese')]
#print(filteredtext)


classificacao = pd.DataFrame({'Classificação': df['Classificação']})
teste = pd.DataFrame({'Teste': df2['Teste']})




"""vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(treinamento['Treinamento'])
modelo = MultinomialNB(alpha=1.0)
modelo.fit(freq_tweets,classificacao['Classificação'])


freq_testes = vectorizer.transform(teste['Teste'])
modelo.predict(freq_testes)

resultados = cross_val_predict(modelo, freq_tweets, classificacao['Classificação'], cv=10)
print(metrics.accuracy_score(classificacao['Classificação'],resultados))

sentimento=['Relevante','Irrelevante']
print(metrics.classification_report(classificacao['Classificação'],resultados,sentimento))

pd.crosstab(classificacao['Classificação'], resultados, rownames=['Real'], colnames=['Predito'], margins=True)"""

treinamento['Treinamento']


AttributeError: 'list' object has no attribute 'split'

In [377]:
xls_file = pd.ExcelFile('Ruffles.xlsx')

df = xls_file.parse('Treinamento')
df2 = xls_file.parse('Teste')


treinamento = pd.DataFrame({'Treinamento': df['Treinamento']})
classificacao = pd.DataFrame({'Classificação': df['Classificação']})
teste = pd.DataFrame({'Teste': df2['Teste']})
                            
treinamento['Tokenized_treinamento'] = treinamento.apply(lambda row: nltk.word_tokenize(row['Treinamento']), axis=1)



stop = stopwords.words('portuguese')

#treinamento = " ".join(re.findall('[A-Z][^A-Z]*', treinamento)) - importante vooos

#tokenized_treinamento = [word_tokenize(palavra) for palavra in treinamento['Treinamento']]
regex = re.compile('[%s]' % re.escape(string.punctuation))

tokenized_treinamento_sem_pontuacao = []

for review in treinamento['Tokenized_treinamento']:
    new_review = []
    for token in review: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)

    tokenized_treinamento_sem_pontuacao.append(new_review)
    
    
    


tokenized_treinamento_sem_stopwords = []
for frase in tokenized_treinamento_sem_pontuacao:
    new_term_vector = []
    for palavra in frase:
        if not palavra in stopwords.words('portuguese'):
            new_term_vector.append(palavra)
    tokenized_treinamento_sem_stopwords.append(new_term_vector)

    
treinamento['Tokenized_treinamento'] = tokenized_treinamento_sem_stopwords



#"".join(treinamento['Tokenized_treinamento'])

#d = MosesDetokenizer()
#detokens = d.detokenize(tokenized_treinamento_sem_stopwords)
#detokens = "".join(detokens)


vectorizer = CountVectorizer(analyzer='word', tokenizer=lambda doc: doc, lowercase=False)
freq_tweets = vectorizer.fit_transform(treinamento['Tokenized_treinamento'])

#freq_tweets = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False).fit_transform(treinamento['Tokenized_treinamento'])
freq_tweets.toarray()
modelo = MultinomialNB(alpha=1)
modelo.fit(freq_tweets,classificacao['Classificação'])


freq_testes = vectorizer.transform(teste['Teste'])
modelo.predict(freq_testes)


resultados = cross_val_predict(modelo, freq_tweets, classificacao['Classificação'], cv=10)
print(metrics.accuracy_score(classificacao['Classificação'],resultados))

sentimento=['Relevante','Irrelevante']
print(metrics.classification_report(classificacao['Classificação'],resultados,sentimento))

pd.crosstab(classificacao['Classificação'], resultados, rownames=['Real'], colnames=['Predito'], margins=True)


teste['Teste'].value_counts()


0.676666666667
             precision    recall  f1-score   support

  Relevante       0.30      0.62      0.40        53
Irrelevante       0.89      0.69      0.78       247

avg / total       0.79      0.68      0.71       300

0      [fvernon, quer, saber, vou, abrir, ruffles, co...
1      [oferecimento, rufflesoficial, motorola, adida...
2      [rt, doreajheniffer, memarialuiza, hunterrowla...
3                       [10, conto, ruffles, quebrei, 💔]
4      [emelyalissa, assassino, é, pessoa, bicho, pap...
5      [rt, loveologyx, vida, adulta, fez, desgostar,...
6                         [104, ruffles, miojo, doritos]
7      [rt, larissakohnlein, vida, igual, pacote, ruf...
8      [fecroft, senti, lá, belíssima, tomando, vinho...
9      [vontade, insana, comer, ruffles, logo, hoje, ...
10                          [ainda, bem, gosto, ruffles]
11                  [to, comendo, ruffles, bem, buenass]
12       [fuckusofmuch, nevermindalot, nao, vai, chegou]
13     [to, aqui, salão, arru

In [380]:
vectorizer = CountVectorizer(ngram_range=(1,2), analyzer='word', tokenizer=lambda doc: doc, lowercase=False)
freq_tweets = vectorizer.fit_transform(treinamento['Tokenized_treinamento'])
modelo = MultinomialNB()
modelo.fit(freq_tweets,classificacao)

freq_testes = vectorizer.transform(teste['Teste'])
modelo.predict(freq_testes)

resultados = cross_val_predict(modelo, freq_tweets, classificacao, cv=10)
#metrics.accuracy_score(classificacao,resultados)

#sentimento=['Relevante','Irrelevante']
#print(metrics.classification_report(classificacao,resultados,sentimento))

C:\Users\taina\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


IndexError: too many indices for array

In [383]:
xls_file = pd.ExcelFile('Ruffles.xlsx')

df = xls_file.parse('Treinamento')
df2 = xls_file.parse('Teste')

treinamento = pd.DataFrame({'Treinamento': df['Treinamento'],
                            'Classificação': df['Classificação']})


teste = pd.DataFrame({'Teste': df2['Teste']})
                            
treinamento['Tokenized_treinamento'] = treinamento.apply(lambda row: nltk.word_tokenize(row['Treinamento']), axis=1)



stop = stopwords.words('portuguese')

#treinamento = " ".join(re.findall('[A-Z][^A-Z]*', treinamento)) - importante vooos

#tokenized_treinamento = [word_tokenize(palavra) for palavra in treinamento['Treinamento']]
regex = re.compile('[%s]' % re.escape(string.punctuation))

tokenized_treinamento_sem_pontuacao = []

for review in treinamento['Tokenized_treinamento']:
    new_review = []
    for token in review: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)

    tokenized_treinamento_sem_pontuacao.append(new_review)
        


tokenized_treinamento_sem_stopwords = []
for frase in tokenized_treinamento_sem_pontuacao:
    new_term_vector = []
    for palavra in frase:
        if not palavra in stopwords.words('portuguese'):
            new_term_vector.append(palavra)
    tokenized_treinamento_sem_stopwords.append(new_term_vector)

    
treinamento['Tokenized_treinamento'] = tokenized_treinamento_sem_stopwords
relevante = treinamento[(treinamento.Classificação == 'Relevante')]
irrelevante = treinamento[(treinamento.Classificação ==  'Irrelevante')]



all_words = []
all_words2 = []
frequencia = {}
frequencia2 = {}
prob_palavra_relevante = {}
prob_palavra_irrelevante = {}


for percorre_tweets in relevante['Tokenized_treinamento']:
    all_words.extend(percorre_tweets)

for percorre_tweets in irrelevante['Tokenized_treinamento']:
    all_words2.extend(percorre_tweets)

for i in all_words:
    frequencia[i] = all_words.count(i)

for i in all_words2:
    frequencia2[i] = all_words2.count(i)   

for j in frequencia:
    prob_palavra_relevante[j] = ((frequencia[j]+1)/(len(all_words) + (len(frequencia)+len(frequencia2))))

for j in frequencia2:
    prob_palavra_irrelevante[j] = ((frequencia2[j]+1)/(len(all_words) + (len(frequencia)+len(frequencia2))))

prob_rel_irr = df.Classificação.value_counts(True)

prob_teste = df2.Classificação.value_counts(True)

#wordlist = nltk.FreqDist(wordlist)
#word_features = all_words.keys()

#print(all_words)

prob_teste

Irrelevante    0.86
Relevante      0.14
Name: Classificação, dtype: float64

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [364]:
teste = pd.DataFrame({'Teste': df2['Teste']})
                            
teste['Tokenized_teste'] = teste.apply(lambda row: nltk.word_tokenize(row['Teste']), axis=1)

relevante_irrelevante = []

tweets = []


for tweet in teste['Tokenized_teste']:
    tweets.append(tweet)

        
def multiply(numbers):  
    total = 1
    for x in numbers:
        total *= x  
    return total

classificacao_teste = []

for i in range(len(tweets)):
    sentenca_relevante = []
    sentenca_irrelevante = []
    irrelevante = []
    for j in tweets[i]:
        if j in frequencia:
            sentenca_relevante.append(((frequencia[j])/(len(all_words)))*prob_rel_irr[1])
        if j not in frequencia:
            sentenca_relevante.append((1/(len(all_words) + (len(frequencia)+len(frequencia2))))*prob_rel_irr[1])
            
        if j in frequencia2:
            sentenca_irrelevante.append(((frequencia2[j])/(len(all_words)))*prob_rel_irr[0])
        if j not in frequencia2:
            sentenca_irrelevante.append((1/(len(all_words) + (len(frequencia)+len(frequencia2))))*prob_rel_irr[0])
        
    
    relevante = multiply(sentenca_relevante)
    irrelevante.append(multiply(sentenca_irrelevante))
    print(irrelevante)
    
    
    
    if irrelevante<relevante:
        classificacao_teste.append('Relevante')
    else:
        classificacao_teste.append('Irrelevante')
    
        




    
    
        
3.02587633966e-58 < 9.55652269161e-77

        
        
            
                                  

    
   

    
    
#prob_palavra_relevante = ((frequencia[j]+1)/(len(all_words) + (len(frequencia)+len(frequencia2))))*prob_rel_irr[1]
#prob_palavra_irrelevante = ((frequencia2[j]+1)/(len(all_words) + (len(frequencia)+len(frequencia2))))*prob_rel_irr[0]
    
    #relevante_irrelevante.append(' '.join(tweet))
    


[1.2195337805982807e-65]
[2.6142246197088749e-47]
[9.9426833603149106e-81]
[1.6761462901521062e-90]
[1.4876590664058873e-63]
[9.9426833603149106e-81]
[1.846019546928247e-33]
[1.2195337805982807e-65]
[4.9246871156242447e-76]
[1.2171663356499402e-38]
[1.4876590664058873e-63]
[1.4876590664058873e-63]
[2.6362992198278426e-38]
[1.4876590664058873e-63]
[2.9912150898816006e-32]
[2.3144145207962853e-33]
[5.9115654556112042e-20]
[1.2195337805982807e-65]
[1.3788109024871128e-39]
[1.1476964928203583e-86]
[1.1624949553858039e-32]
[1.304632629681158e-32]
[1.6148439573838434e-37]
[1.2195337805982807e-65]
[8.9708089431401581e-12]
[9.9145706809560798e-54]
[1.4210633941830906e-39]
[2.1792814233979742e-84]
[7.692383507433781e-19]
[3.8800412353123243e-67]
[2.2860327024561474e-55]
[1.7207258992838778e-46]
[1.2195337805982807e-65]
[3.5411751542870352e-51]
[4.7921995361493059e-32]
[1.2195337805982807e-65]
[9.0037766427087171e-58]
[5.418408770803415e-56]
[1.4876590664058873e-63]
[2.9539916006198276e-24]
[5.9

False

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
